# Summarization (PyTorch)


*   https://huggingface.co/learn/nlp-course/chapter7/5?fw=pt



Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

You will need to setup git, adapt your email and name in the following cell.

In [2]:
'''!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"'''

'!git config --global user.email "you@example.com"\n!git config --global user.name "Your Name"'

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [3]:
'''from huggingface_hub import notebook_login

notebook_login()'''

'from huggingface_hub import notebook_login\n\nnotebook_login()'

In [4]:
'''#from huggingface_hub.repocard import yaml
from datasets import load_dataset, Dataset, load_metric
import pandas as pd
import csv
import torch
import pandas as pd
import json

def dataframe_to_jsonl(dataframe, jsonl_file_path):
    with open(jsonl_file_path, 'w') as jsonl_file:
        for _, row in dataframe.iterrows():
            json_object = row.to_json(orient='columns')
            jsonl_file.write(json_object + '\n')

colnames = ['sentence', 'label', 'data', 'regex', 'subset']
train_df = pd.read_csv("COVID_Train_Set.csv", header=0, index_col=None, names=colnames, encoding="utf-8", delimiter=",", quoting=csv.QUOTE_NONE, error_bad_lines=False, skiprows=1, engine='python', na_values='None')
train_df
#train_df = train_df.loc[train_df["sentence"] != 'None']
train_df = train_df.dropna()
train_df['regex'] = train_df['regex'].map({True: 'True', False: 'False'})
train_df['data'] = train_df['data'].astype(object)
train_df['language'] = 'en'
print(train_df)
print(train_df.iloc[:5])
test_df = pd.read_csv("COVID_Test_Set.csv", header=0, index_col=None, names=colnames, encoding="utf-8", delimiter=",", quoting=csv.QUOTE_NONE, error_bad_lines=False, skiprows=1, engine='python', na_values='None')
test_df = test_df.dropna()
#test_df = test_df.loc[test_df["sentence"] != 'None']
test_df['regex'] = test_df['regex'].map({True: 'True', False: 'False'})
test_df['data'] = test_df['data'].astype(object)
test_df['language'] = 'en'
print(test_df)
print(test_df.iloc[:5])

# Example usage
  # Replace 'data.csv' with your DataFrame source
jsonl_file_path = 'train.jsonl'
dataframe_to_jsonl(train_df, jsonl_file_path)

jsonl_file_path = 'test.jsonl'
dataframe_to_jsonl(test_df, jsonl_file_path)

data_load = {"train": "train.jsonl", "test": "test.jsonl"}
# TypeError: expected str, bytes or os.PathLike object, not Dataset
#covid_dataset = load_dataset("json", data_files=data_load)
'''

'#from huggingface_hub.repocard import yaml\nfrom datasets import load_dataset, Dataset, load_metric\nimport pandas as pd\nimport csv\nimport torch\nimport pandas as pd\nimport json\n\ndef dataframe_to_jsonl(dataframe, jsonl_file_path):\n    with open(jsonl_file_path, \'w\') as jsonl_file:\n        for _, row in dataframe.iterrows():\n            json_object = row.to_json(orient=\'columns\')\n            jsonl_file.write(json_object + \'\n\')\n\ncolnames = [\'sentence\', \'label\', \'data\', \'regex\', \'subset\']\ntrain_df = pd.read_csv("COVID_Train_Set.csv", header=0, index_col=None, names=colnames, encoding="utf-8", delimiter=",", quoting=csv.QUOTE_NONE, error_bad_lines=False, skiprows=1, engine=\'python\', na_values=\'None\')\ntrain_df\n#train_df = train_df.loc[train_df["sentence"] != \'None\']\ntrain_df = train_df.dropna()\ntrain_df[\'regex\'] = train_df[\'regex\'].map({True: \'True\', False: \'False\'})\ntrain_df[\'data\'] = train_df[\'data\'].astype(object)\ntrain_df[\'language\

In [5]:
'''from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig, AutoModel, pipeline

 # Wrong code
src = "en"
dst = "es"

task_name = f"translation_{src}_to_{dst}"
model_name = f"Helsinki-NLP/opus-mt-{src}-{dst}"
device = torch.device("cuda:0")
#bart_config = BartConfig(
#    vocab_size=vocab_size, d_model=64, encoder_ffn_dim=64, encoder_layers=2, decoder_ffn_dim=64, decoder_layers=2
#)
#config = bart_config.to_dict()
#model = AutoModelForSeq2SeqLM.from_config(BartConfig.from_dict(config))
translator  = pipeline(task_name, model=model_name, tokenizer=model_name, device=device)
#vocab_size = 99000
#num_new_tokens = 10
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")
#model.resize_token_embeddings(model.config.vocab_size + num_new_tokens)
config = AutoConfig.from_pretrained("Helsinki-NLP/opus-mt-en-es")

print(model.config)
model.config.vocab_size
#print(vocab_size)'''

'from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig, AutoModel, pipeline\n\n # Wrong code\nsrc = "en"\ndst = "es"\n\ntask_name = f"translation_{src}_to_{dst}"\nmodel_name = f"Helsinki-NLP/opus-mt-{src}-{dst}"\ndevice = torch.device("cuda:0")\n#bart_config = BartConfig(\n#    vocab_size=vocab_size, d_model=64, encoder_ffn_dim=64, encoder_layers=2, decoder_ffn_dim=64, decoder_layers=2\n#)\n#config = bart_config.to_dict()\n#model = AutoModelForSeq2SeqLM.from_config(BartConfig.from_dict(config))\ntranslator  = pipeline(task_name, model=model_name, tokenizer=model_name, device=device)\n#vocab_size = 99000\n#num_new_tokens = 10\ntokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")\n\nmodel = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")\n#model.resize_token_embeddings(model.config.vocab_size + num_new_tokens)\nconfig = AutoConfig.from_pretrained("Helsinki-NLP/opus-mt-en-es")\n\nprint(model.config)\nmodel.config.vocab_size\n#prin

In [6]:
'''import pandas as pd
#df = pd.read_csv("21-1461.csv", encoding="utf-8", index_col=0)
#print(df)
#x = df.to_string(header=True, index=True)
#print(x)
train_total = train_df.shape[0] - 1
print(train_total)
train_sen_pal = train_df['sentence'].values.tolist()
print(train_sen_pal)
train_dat_pal = train_df['data'].values.tolist()
print(train_dat_pal)
test_total = test_df.shape[0] - 1
print(test_total)
test_sen_pal = test_df['sentence'].values.tolist()
print(test_sen_pal)
test_dat_pal = test_df['data'].values.tolist()
print(test_dat_pal)

def classify_text(data):
    text = data["sentence"]
    result = classifier(text, labels)
    return result["labels"][0]

#data2 = {'text': [pal]}
'''
'''
#new_list = [item.split() for item in pal]
#print(new_list)
#lst = pal
#s = "".join(lst)
#result = s.split('!')
#print(result)
#myList = [i.split('\t')[0] for i in pal]
#print(pal)
#for pal in df['sentence']:
#df['lang_ru'] = df['sentence'].apply(lambda pal :translator(pal, max_length=65990))
#data = data.applymap(lambda pal: translator(pal)[0]['translation_text'])
train_df['sentence_es'] = train_df['sentence'].apply(lambda train_sen_pal :translator(train_sen_pal)[0]['translation_text'])
train_df['data_es'] = train_df['data'].apply(lambda train_dat_pal :translator(train_dat_pal)[0]['translation_text'])
test_df['sentence_es'] = test_df['sentence'].apply(lambda test_sen_pal :translator(test_sen_pal)[0]['translation_text'])
test_df['data_es'] = test_df['data'].apply(lambda test_dat_pal :translator(test_dat_pal)[0]['translation_text'])
#data["sentiment"] = data["sentence"].apply(lambda pal: classifier(pal)[0]['label'])
#data["zero-shot"] = data["sentence"].apply(lambda pal: zsclassifier(pal, labels)["scores"])
#data["generated"] = data["sentence"].apply(lambda pal: generator(pal, max_length=50)[0]["generated_text"])
print(train_df)
print(test_df)
train_df.to_csv("COVID_Train_Set_Preset.csv")
test_df.to_csv("COVID_Test_Set_Preset.csv")
#print(df)

#translated = translator(vals).get('translation_text')
#print(translated)
#for x in df['sentence']:
# print(translator(i))
#df['lang-ru'] = [translator(i) for x]
#print(df)
#df.to_csv('Translated_column.csv')'''

'\n#new_list = [item.split() for item in pal]\n#print(new_list)\n#lst = pal\n#s = "".join(lst)\n#result = s.split(\'!\')\n#print(result)\n#myList = [i.split(\'\t\')[0] for i in pal]\n#print(pal)\n#for pal in df[\'sentence\']:\n#df[\'lang_ru\'] = df[\'sentence\'].apply(lambda pal :translator(pal, max_length=65990))\n#data = data.applymap(lambda pal: translator(pal)[0][\'translation_text\'])\ntrain_df[\'sentence_es\'] = train_df[\'sentence\'].apply(lambda train_sen_pal :translator(train_sen_pal)[0][\'translation_text\'])\ntrain_df[\'data_es\'] = train_df[\'data\'].apply(lambda train_dat_pal :translator(train_dat_pal)[0][\'translation_text\'])\ntest_df[\'sentence_es\'] = test_df[\'sentence\'].apply(lambda test_sen_pal :translator(test_sen_pal)[0][\'translation_text\'])\ntest_df[\'data_es\'] = test_df[\'data\'].apply(lambda test_dat_pal :translator(test_dat_pal)[0][\'translation_text\'])\n#data["sentiment"] = data["sentence"].apply(lambda pal: classifier(pal)[0][\'label\'])\n#data["zero-sh

In [7]:
#from huggingface_hub.repocard import yaml
from datasets import load_dataset, Dataset, load_metric
import pandas as pd
import csv
import torch
import pandas as pd
import json

def dataframe_to_jsonl(dataframe, jsonl_file_path):
    with open(jsonl_file_path, 'w') as jsonl_file:
        for _, row in dataframe.iterrows():
            json_object = row.to_json(orient='columns')
            jsonl_file.write(json_object + '\n')

colnames = ['sentence', 'label', 'data', 'regex', 'subset', 'language']
train_df = pd.read_csv("COVID_Train_Set_English-Spanish.csv", header=0, index_col=None, names=colnames, encoding="utf-8", delimiter=",", quoting=csv.QUOTE_NONE, error_bad_lines=False, skiprows=1, engine='python', na_values='None')
train_df
#train_df = train_df.loc[train_df["sentence"] != 'None']
train_df = train_df.dropna()
train_df['regex'] = train_df['regex'].map({True: 'True', False: 'False'})
train_df['data'] = train_df['data'].astype(object)
#train_df['sentence'] = ['sentence'] + ['lang_es']
print(train_df)
print(train_df.iloc[:5])
test_df = pd.read_csv("COVID_Test_Set_English-Spanish.csv", header=0, index_col=None, names=colnames, encoding="utf-8", delimiter=",", quoting=csv.QUOTE_NONE, error_bad_lines=False, skiprows=1, engine='python', na_values='None')
test_df = test_df.dropna()
#test_df = test_df.loc[test_df["sentence"] != 'None']
test_df['regex'] = test_df['regex'].map({True: 'True', False: 'False'})
test_df['data'] = test_df['data'].astype(object)
#test_df['sentence'] = ['sentence'] + ['lang_es']
print(test_df)
print(test_df.iloc[:5])

# Example usage
  # Replace 'data.csv' with your DataFrame source
jsonl_file_path = 'train.jsonl'
dataframe_to_jsonl(train_df, jsonl_file_path)

jsonl_file_path = 'test.jsonl'
dataframe_to_jsonl(test_df, jsonl_file_path)

data_load = {"train": "train.jsonl", "test": "test.jsonl"}
# TypeError: expected str, bytes or os.PathLike object, not Dataset
covid_dataset = load_dataset("json", data_files=data_load)
print(covid_dataset)

<ipython-input-7-0eee3df1e5d3>:16: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  train_df = pd.read_csv("COVID_Train_Set_English-Spanish.csv", header=0, index_col=None, names=colnames, encoding="utf-8", delimiter=",", quoting=csv.QUOTE_NONE, error_bad_lines=False, skiprows=1, engine='python', na_values='None')
Skipping line 2848: Expected 6 fields in line 2848, saw 7
Skipping line 2862: Expected 6 fields in line 2862, saw 7
Skipping line 2864: Expected 6 fields in line 2864, saw 7
Skipping line 2865: Expected 6 fields in line 2865, saw 7
Skipping line 2917: Expected 6 fields in line 2917, saw 8
Skipping line 2921: Expected 6 fields in line 2921, saw 7
Skipping line 2951: Expected 6 fields in line 2951, saw 7
Skipping line 3019: Expected 6 fields in line 3019, saw 7
Skipping line 3097: Expected 6 fields in line 3097, saw 7
Skipping line 3184: Expected 6 fields in line 3184, saw 7
Skipping line

                                               sentence  label        data  \
0                SARS-CoV-2 Omicron Variant of Concern      1  SARS-CoV-2   
1                SARS-CoV-2 Omicron Variant of Concern      1  SARS-CoV-2   
2     Table 2. Monoclonal Antibodies for Treatment o...      1    COVID-19   
3     02/11/2022 Mild-to-moderate COVID-19 in adults...      1    COVID-19   
4     positive results who are at high risk for prog...      1    positive   
...                                                 ...    ...         ...   
5305  PALABRAS CLAVE Ciencia de la confianza; confia...      1      social   
5306  También haré eco de lo que creo que Gil dijo i...      1   comunidad   
5307  Deficiencias existentes en la investigación/Ár...      1        Zona   
5308  [5] Schiavo R. 2021. Desafiando la desinformac...      1   comunidad   
5309  [17] Shapin S. Una historia social de la verda...      1      social   

     regex subset language  
0     True  Train       en  
1    

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-4ff7efe4c8fca509/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'data', 'regex', 'subset', 'language'],
        num_rows: 5310
    })
    test: Dataset({
        features: ['sentence', 'label', 'data', 'regex', 'subset', 'language'],
        num_rows: 1503
    })
})


In [8]:
#from datasets import load_dataset
# Load the dataset using load_dataset from the datasets library
covid_dataset = load_dataset("json", data_files=data_load)

# Access the subsets for English and Spanish
english_dataset = covid_dataset.filter(lambda example: example["language"] == "en")
spanish_dataset = covid_dataset.filter(lambda example: example["language"] == "es")

# Print the English dataset
print(english_dataset)

# Print the Spanish dataset
print(spanish_dataset)


  0%|          | 0/2 [00:00<?, ?it/s]

Filter:   0%|          | 0/5310 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1503 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5310 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1503 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'data', 'regex', 'subset', 'language'],
        num_rows: 2699
    })
    test: Dataset({
        features: ['sentence', 'label', 'data', 'regex', 'subset', 'language'],
        num_rows: 765
    })
})
DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'data', 'regex', 'subset', 'language'],
        num_rows: 2611
    })
    test: Dataset({
        features: ['sentence', 'label', 'data', 'regex', 'subset', 'language'],
        num_rows: 738
    })
})


In [9]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Data: {example['data']}'")
        print(f"'>> Sentence: {example['sentence']}'")


show_samples(english_dataset)


'>> Data: disease'
'>> Sentence: Chronic kidney disease (CKD) on dialysis'

'>> Data: disease'
'>> Sentence: Table 2 Reasons for referral of inpatients without previous immune-m­ ediated inflammatory disease (IMID) according to the discharge diagnosis of IMID or other diseases'

'>> Data: Clinical'
'>> Sentence: Luminal Clinical'


In [10]:
'''from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

#raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True, padding=True)


tokenized_datasets = covid_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)'''

'from datasets import load_dataset\nfrom transformers import AutoTokenizer, DataCollatorWithPadding\n\n#raw_datasets = load_dataset("glue", "mrpc")\ncheckpoint = "bert-base-uncased"\ntokenizer = AutoTokenizer.from_pretrained(checkpoint)\n\ndef tokenize_function(example):\n    return tokenizer(example["sentence"], truncation=True, padding=True)\n\n\ntokenized_datasets = covid_dataset.map(tokenize_function, batched=True)\ndata_collator = DataCollatorWithPadding(tokenizer=tokenizer)'

In [11]:
english_dataset.set_format("pandas")
english_df = english_dataset["train"][:]
# Show counts for top 20 products
english_df["data"].value_counts()[:20]

COVID-19       752
disease        191
SARS-CoV-2     146
vaccine        141
case           131
clinical       126
Clinical       124
trial           66
Disease         66
Vaccine         66
Case            65
flu             64
pandemic        57
covid-19        42
positive        41
Social          39
social          39
personal        32
Negative        31
coronavirus     31
Name: data, dtype: int64

In [12]:
'''def filter_keywords(example):
    return (
        example["data"] == "COVID-19"
        or example["data"] == "SARS-Cov-2"
    )'''

'def filter_keywords(example):\n    return (\n        example["data"] == "COVID-19"\n        or example["data"] == "SARS-Cov-2"\n    )'

In [13]:
english_dataset.reset_format()

In [14]:
show_samples(english_dataset)
show_samples(spanish_dataset)


'>> Data: disease'
'>> Sentence: Chronic kidney disease (CKD) on dialysis'

'>> Data: disease'
'>> Sentence: Table 2 Reasons for referral of inpatients without previous immune-m­ ediated inflammatory disease (IMID) according to the discharge diagnosis of IMID or other diseases'

'>> Data: Clinical'
'>> Sentence: Luminal Clinical'

'>> Data: cluster'
'>> Sentence: 396  Asociación de grupos de medicamentos y marginación del mercado laboral en jóvenes refugiados y nacidos en Suecia con trastornos mentales comunes en Suecia'

'>> Data: SARS-CoV-2'
'>> Sentence: """SARS-CoV-2 y EBV; el costo de un segundo mitocondrial """"whammy""""?"""'

'>> Data: COVID-19'
'>> Sentence: 4. CDC. COVID-19 Vacunación. Centros de Control y Prevención de Enfermedades. 2021 [cite ́ 13 mai 2022].'


In [15]:
'''spanish_popular = spanish_traindata.filter(filter_keywords)
english_popular = english_traindata.filter(filter_keywords)
show_samples(english_popular)'''

'spanish_popular = spanish_traindata.filter(filter_keywords)\nenglish_popular = english_traindata.filter(filter_keywords)\nshow_samples(english_popular)'

In [16]:
'''from datasets import concatenate_datasets, DatasetDict

# Define english_popular as a DatasetDict
#english_popular = DatasetDict()

# Assuming you have defined English popular datasets for different splits (e.g., "train", "validation", "test")
#english_popular["train"] = english_traindata
#english_popular["validation"] = english_validation_dataset
#english_popular["test"] = english_testdata

# Similarly, define spanish_popular as a DatasetDict if you have Spanish popular datasets for the same splits
#for split in english_dataset.keys():
# Create a new DatasetDict called popular_dataset and concatenate English and Spanish datasets
#popular_dataset = DatasetDict()

popular_dataset = concatenate_datasets([english_dataset, spanish_dataset])

popular_dataset= popular_dataset.shuffle(seed=42)
'''
'''
# Function to show a few samples from the dataset
def show_samples(dataset):
    for split in dataset.keys():
        print(f"Split: {split}")
        samples = dataset[split][:5]  # Change the number (5) to adjust the number of samples to display
        for sample in samples:
            print(sample)
        print("--------")
'''
'''
# Peek at a few examples
show_samples(popular_dataset)
print(popular_dataset)
'''

'\n# Peek at a few examples\nshow_samples(popular_dataset)\nprint(popular_dataset)\n'

In [17]:
'''from datasets import concatenate_datasets, DatasetDict

# Assuming you have defined English and Spanish datasets named 'english_dataset' and 'spanish_dataset'

# Define the split names for each dataset
english_splits = {'train', 'test'}
spanish_splits = {'train', 'test'}

# Create DatasetDicts for English and Spanish datasets
english_datasets = DatasetDict({split: english_dataset[split] for split in english_splits})
spanish_datasets = DatasetDict({split: spanish_dataset[split] for split in spanish_splits})

# Create a new DatasetDict called popular_dataset and concatenate English and Spanish datasets
popular_dataset = concatenate_datasets([english_datasets, spanish_datasets])

# Shuffle the popular_dataset
popular_dataset = popular_dataset.shuffle(seed=42)

# Function to show a few samples from the dataset
def show_samples(dataset):
    for split in dataset.keys():
        print(f"Split: {split}")
        samples = dataset[split][:5]  # Change the number (5) to adjust the number of samples to display
        for sample in samples:
            print(sample)
        print("--------")

# Peek at a few examples
show_samples(popular_dataset)

print(popular_dataset)
'''

'from datasets import concatenate_datasets, DatasetDict\n\n# Assuming you have defined English and Spanish datasets named \'english_dataset\' and \'spanish_dataset\'\n\n# Define the split names for each dataset\nenglish_splits = {\'train\', \'test\'}\nspanish_splits = {\'train\', \'test\'}\n\n# Create DatasetDicts for English and Spanish datasets\nenglish_datasets = DatasetDict({split: english_dataset[split] for split in english_splits})\nspanish_datasets = DatasetDict({split: spanish_dataset[split] for split in spanish_splits})\n\n# Create a new DatasetDict called popular_dataset and concatenate English and Spanish datasets\npopular_dataset = concatenate_datasets([english_datasets, spanish_datasets])\n\n# Shuffle the popular_dataset\npopular_dataset = popular_dataset.shuffle(seed=42)\n\n# Function to show a few samples from the dataset\ndef show_samples(dataset):\n    for split in dataset.keys():\n        print(f"Split: {split}")\n        samples = dataset[split][:5]  # Change the num

In [20]:
'''from datasets import concatenate_datasets, DatasetDict

# Assuming you have defined English and Spanish datasets named 'english_dataset' and 'spanish_dataset'

# Define the split names for each dataset
english_splits = {'train', 'test'}
spanish_splits = {'train', 'test'}

# Create DatasetDicts for English and Spanish datasets
english_datasets = DatasetDict({split: english_dataset[split] for split in english_splits})
spanish_datasets = DatasetDict({split: spanish_dataset[split] for split in spanish_splits})

# Select a common split to interleave the datasets (e.g., "train" split)
common_split = 'train'

# Create a new DatasetDict called popular_dataset and concatenate English and Spanish datasets
popular_dataset = DatasetDict()
popular_dataset[common_split] = concatenate_datasets([english_datasets[common_split], spanish_datasets[common_split]])

# Shuffle the popular_dataset
popular_dataset[common_split] = popular_dataset[common_split].shuffle(seed=42)

# Function to show a few samples from the dataset
def show_samples(dataset):
    for split in dataset.keys():
        print(f"Split: {split}")
        samples = dataset[split][:5]  # Change the number (5) to adjust the number of samples to display
        for sample in samples:
            print(sample)
        print("--------")

# Peek at a few examples
show_samples(popular_dataset)

print(popular_dataset)'''


'from datasets import concatenate_datasets, DatasetDict\n\n# Assuming you have defined English and Spanish datasets named \'english_dataset\' and \'spanish_dataset\'\n\n# Define the split names for each dataset\nenglish_splits = {\'train\', \'test\'}\nspanish_splits = {\'train\', \'test\'}\n\n# Create DatasetDicts for English and Spanish datasets\nenglish_datasets = DatasetDict({split: english_dataset[split] for split in english_splits})\nspanish_datasets = DatasetDict({split: spanish_dataset[split] for split in spanish_splits})\n\n# Select a common split to interleave the datasets (e.g., "train" split)\ncommon_split = \'train\'\n\n# Create a new DatasetDict called popular_dataset and concatenate English and Spanish datasets\npopular_dataset = DatasetDict()\npopular_dataset[common_split] = concatenate_datasets([english_datasets[common_split], spanish_datasets[common_split]])\n\n# Shuffle the popular_dataset\npopular_dataset[common_split] = popular_dataset[common_split].shuffle(seed=42)

In [21]:
'''# Select a common split to interleave the datasets (e.g., "train" split)
common_split = 'test'

# Create a new DatasetDict called popular_dataset and concatenate English and Spanish datasets
popular_dataset = DatasetDict()
popular_dataset[common_split] = concatenate_datasets([english_datasets[common_split], spanish_datasets[common_split]])

# Shuffle the popular_dataset
popular_dataset[common_split] = popular_dataset[common_split].shuffle(seed=42)'''

'# Select a common split to interleave the datasets (e.g., "train" split)\ncommon_split = \'test\'\n\n# Create a new DatasetDict called popular_dataset and concatenate English and Spanish datasets\npopular_dataset = DatasetDict()\npopular_dataset[common_split] = concatenate_datasets([english_datasets[common_split], spanish_datasets[common_split]])\n\n# Shuffle the popular_dataset\npopular_dataset[common_split] = popular_dataset[common_split].shuffle(seed=42)'

In [22]:
from datasets import concatenate_datasets, DatasetDict

language_dataset = DatasetDict()

for split in english_dataset.keys():
   language_dataset[split] = concatenate_datasets(
        [english_dataset[split], spanish_dataset[split]]
        )
   language_dataset[split] = language_dataset[split].shuffle(seed=42)

# Peek at a few examples
show_samples(language_dataset)


'>> Data: Positivo'
'>> Sentence: P066 Psoriasis asociada con el interferón Be^ta-1b en un paciente con esclerosis múltiple con redesafio positivo'

'>> Data: Coronavirus'
'>> Sentence: 7. Sood S. Psychological effects of the Coronavirus disease-2019 pandemic. Res Human Med Educ 2020; 7 (11): 23-26.'

'>> Data: Respiratorio'
'>> Sentence: Infecciones del tracto respiratorio y COVID-19'


In [23]:
language_dataset = language_dataset.filter(lambda x: len(x["sentence"].split()) > 2)

Filter:   0%|          | 0/5310 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1503 [00:00<?, ? examples/s]

In [24]:
'''books_dataset = books_dataset.filter(lambda x: len(x["review_title"].split()) > 2)'''

'books_dataset = books_dataset.filter(lambda x: len(x["review_title"].split()) > 2)'

In [25]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [26]:
sentences1 = train_df.iloc[0, 0]
example1 = "".join(sentences1)

sentences2 = train_df.iloc[2, 0]
example2 = "".join(sentences2)

sentences3 = train_df.iloc[4, 0]
example3 = "".join(sentences3)


In [27]:
inputs = tokenizer(example1)
inputs

{'input_ids': [320, 50922, 264, 10305, 561, 4959, 443, 210009, 70372, 304, 157402, 272, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [28]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁S',
 'ARS',
 '-',
 'Co',
 'V',
 '-2',
 '▁O',
 'micron',
 '▁Variant',
 '▁of',
 '▁Concer',
 'n',
 '</s>']

In [29]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["sentence"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["data"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [30]:
tokenized_datasets = language_dataset.map(preprocess_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/4859 [00:00<?, ? examples/s]

Map:   0%|          | 0/1387 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'data', 'regex', 'subset', 'language', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4859
    })
    test: Dataset({
        features: ['sentence', 'label', 'data', 'regex', 'subset', 'language', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1387
    })
})

In [31]:
generated_summary = example2
reference_summary = example3

In [32]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b4fbc3c87dcbd089247de1974432176c77785d053a4070ae396c0fa370751111
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [33]:
import evaluate

rouge_score = evaluate.load("rouge")

In [34]:
scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

{'rouge1': 0.09090909090909091,
 'rouge2': 0.0,
 'rougeL': 0.09090909090909091,
 'rougeLsum': 0.09090909090909091}

In [35]:
scores["rouge1"]

0.09090909090909091

In [36]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [38]:
'''from nltk.tokenize import sent_tokenize


def three_sentence_summary(sentence):
    return "\n".join(sent_tokenize(sentence)[:3])


print(three_sentence_summary(train_dataset["sentence"]))'''

'from nltk.tokenize import sent_tokenize\n\n\ndef three_sentence_summary(sentence):\n    return "\n".join(sent_tokenize(sentence)[:3])\n\n\nprint(three_sentence_summary(train_dataset["sentence"]))'

In [39]:
'''from nltk.tokenize import sent_tokenize

def three_sentence_summary(sentence):
    return "\n".join(sent_tokenize(sentence)[:3])

# Assuming you have a dataset named 'train_dataset' with a column named 'sentence'
# Replace 'train_dataset["sentence"]' with the actual column name in your dataset
sentence_column = train_dataset["sentence"]

# Apply the three_sentence_summary function to each sentence in the column
summary_column = sentence_column.apply(three_sentence_summary)

# Print the summaries
print(summary_column)'''

'from nltk.tokenize import sent_tokenize\n\ndef three_sentence_summary(sentence):\n    return "\n".join(sent_tokenize(sentence)[:3])\n\n# Assuming you have a dataset named \'train_dataset\' with a column named \'sentence\'\n# Replace \'train_dataset["sentence"]\' with the actual column name in your dataset\nsentence_column = train_dataset["sentence"]\n\n# Apply the three_sentence_summary function to each sentence in the column\nsummary_column = sentence_column.apply(three_sentence_summary)\n\n# Print the summaries\nprint(summary_column)'

In [40]:
'''from nltk.tokenize import sent_tokenize
import pandas as pd

def three_sentence_summary(sentence):
    return "\n".join(sent_tokenize(sentence)[:3])

# Assuming you have a list named 'train_dataset' with a column named 'sentence'
# Replace 'train_dataset["sentence"]' with the actual column name in your dataset
sentence_column = train_dataset["sentence"]

# Convert the list to a DataFrame
train_df = pd.DataFrame({"sentence": sentence_column})

# Apply the three_sentence_summary function to each sentence in the column
train_df["summary"] = train_df["sentence"].apply(three_sentence_summary)

# Print the summaries
print(train_df["summary"])'''


'from nltk.tokenize import sent_tokenize\nimport pandas as pd\n\ndef three_sentence_summary(sentence):\n    return "\n".join(sent_tokenize(sentence)[:3])\n\n# Assuming you have a list named \'train_dataset\' with a column named \'sentence\'\n# Replace \'train_dataset["sentence"]\' with the actual column name in your dataset\nsentence_column = train_dataset["sentence"]\n\n# Convert the list to a DataFrame\ntrain_df = pd.DataFrame({"sentence": sentence_column})\n\n# Apply the three_sentence_summary function to each sentence in the column\ntrain_df["summary"] = train_df["sentence"].apply(three_sentence_summary)\n\n# Print the summaries\nprint(train_df["summary"])'

In [41]:
'''from nltk.tokenize import sent_tokenize
import pandas as pd

def three_sentence_summary(sentence):
    return "\n".join(sent_tokenize(sentence)[:3])

# Assuming you have a DataFrame named 'train_df' with a column named 'sentence'
# Replace 'train_df["sentence"]' with the actual column name in your dataset
sentence_column = train_df["sentence"]

# Convert the DataFrame column to a list
sentence_list = sentence_column.tolist()

# Apply the three_sentence_summary function to each sentence in the list
summary_list = [three_sentence_summary(sentence) for sentence in sentence_list]

# Create a new DataFrame with the 'sentence' and 'summary' columns
summary_df = pd.DataFrame({"sentence": sentence_list, "summary": summary_list})

# Print the summaries
print(summary_df["summary"])'''


'from nltk.tokenize import sent_tokenize\nimport pandas as pd\n\ndef three_sentence_summary(sentence):\n    return "\n".join(sent_tokenize(sentence)[:3])\n\n# Assuming you have a DataFrame named \'train_df\' with a column named \'sentence\'\n# Replace \'train_df["sentence"]\' with the actual column name in your dataset\nsentence_column = train_df["sentence"]\n\n# Convert the DataFrame column to a list\nsentence_list = sentence_column.tolist()\n\n# Apply the three_sentence_summary function to each sentence in the list\nsummary_list = [three_sentence_summary(sentence) for sentence in sentence_list]\n\n# Create a new DataFrame with the \'sentence\' and \'summary\' columns\nsummary_df = pd.DataFrame({"sentence": sentence_list, "summary": summary_list})\n\n# Print the summaries\nprint(summary_df["summary"])'

In [42]:
'''from nltk.tokenize import sent_tokenize
import pandas as pd

def three_sentence_summary(sentence):
    return "\n".join(sent_tokenize(sentence)[:3])

# Assuming you have a DataFrame named 'train_df' with a column named 'sentences'
# Replace 'train_df["sentences"]' with the actual column name in your dataset
sentence_column = train_df["sentence"]

# Convert the DataFrame column to a list
sentence_list = sentence_column.tolist()

# Apply the three_sentence_summary function to each sentence in the list
summary_list = [three_sentence_summary(sentence) for sentence in sentence_list]

# Create a new DataFrame with the 'sentence' and 'summary' columns
summary_df = pd.DataFrame({"sentence": sentence_list, "summary": summary_list})

# Print the summaries
print(summary_df["summary"])

def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(sentence) for sentence in train_dataset["sentence"]]
    return metric.compute(predictions=summaries, references=train_dataset["data"])

score = evaluate_baseline(train_dataset, rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)
rouge_dict
'''

'from nltk.tokenize import sent_tokenize\nimport pandas as pd\n\ndef three_sentence_summary(sentence):\n    return "\n".join(sent_tokenize(sentence)[:3])\n\n# Assuming you have a DataFrame named \'train_df\' with a column named \'sentences\'\n# Replace \'train_df["sentences"]\' with the actual column name in your dataset\nsentence_column = train_df["sentence"]\n\n# Convert the DataFrame column to a list\nsentence_list = sentence_column.tolist()\n\n# Apply the three_sentence_summary function to each sentence in the list\nsummary_list = [three_sentence_summary(sentence) for sentence in sentence_list]\n\n# Create a new DataFrame with the \'sentence\' and \'summary\' columns\nsummary_df = pd.DataFrame({"sentence": sentence_list, "summary": summary_list})\n\n# Print the summaries\nprint(summary_df["summary"])\n\ndef evaluate_baseline(dataset, metric):\n    summaries = [three_sentence_summary(sentence) for sentence in train_dataset["sentence"]]\n    return metric.compute(predictions=summarie

In [43]:
from nltk.tokenize import sent_tokenize
import pandas as pd

def three_sentence_summary(sentence):
    return "\n".join(sent_tokenize(sentence)[:3])

# Assuming you have a DataFrame named 'train_df' with a column named 'sentence'
# Replace 'train_df["sentence"]' with the actual column name in your dataset
sentence_column = train_df["sentence"]

# Convert the DataFrame column to a list
sentence_list = sentence_column.tolist()

# Apply the three_sentence_summary function to each sentence in the list
summary_list = [three_sentence_summary(sentence) for sentence in sentence_list]

# Create a new DataFrame with the 'sentence' and 'summary' columns
summary_df = pd.DataFrame({"sentence": sentence_list, "summary": summary_list})

# Print the summaries
print(summary_df["summary"])

def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(sentence) for sentence in dataset["sentence"]]
    return metric.compute(predictions=summaries, references=dataset["data"])

score = evaluate_baseline(language_dataset["train"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names)
rouge_dict


0                  SARS-CoV-2 Omicron Variant of Concern
1                  SARS-CoV-2 Omicron Variant of Concern
2       Table 2.\nMonoclonal Antibodies for Treatment ...
3       02/11/2022 Mild-to-moderate COVID-19 in adults...
4       positive results who are at high risk for prog...
                              ...                        
5305    PALABRAS CLAVE Ciencia de la confianza; confia...
5306    También haré eco de lo que creo que Gil dijo i...
5307    Deficiencias existentes en la investigación/Ár...
5308    [5] Schiavo R. 2021.\nDesafiando la desinforma...
5309    [17] Shapin S. Una historia social de la verda...
Name: summary, Length: 5310, dtype: object


{'rouge1': 15.7, 'rouge2': 6.32, 'rougeL': 15.71, 'rougeLsum': 15.71}

In [44]:
'''from nltk.tokenize import sent_tokenize
def three_sentence_summary(sentence):
  sentences = sent_tokenize(sentence)
  summary = "\n".join(sentences[:3])
  return summary'''

'from nltk.tokenize import sent_tokenize\ndef three_sentence_summary(sentence):\n  sentences = sent_tokenize(sentence)\n  summary = "\n".join(sentences[:3])\n  return summary'

In [45]:
'''sentence_column = train_dataset["sentence"]'''

'sentence_column = train_dataset["sentence"]'

In [46]:
'''summary_column = sentence_column.apply(three_sentence_summary)'''

'summary_column = sentence_column.apply(three_sentence_summary)'

In [47]:
'''for summary in summary_column:
print(summary)'''

'for summary in summary_column:\nprint(summary)'

In [48]:
'''def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(sentence) for text in covid_dataset["sentence"]]
    return metric.compute(predictions=summaries, references=covid_dataset["data"])'''

'def evaluate_baseline(dataset, metric):\n    summaries = [three_sentence_summary(sentence) for text in covid_dataset["sentence"]]\n    return metric.compute(predictions=summaries, references=covid_dataset["data"])'

In [49]:
'''import pandas as pd

score = evaluate_baseline(train_dataset, rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)
rouge_dict'''


'import pandas as pd\n\nscore = evaluate_baseline(train_dataset, rouge_score)\nrouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]\nrouge_dict = dict((rn, round(score[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)\nrouge_dict'

In [50]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [51]:
'''from huggingface_hub import notebook_login

notebook_login()'''

'from huggingface_hub import notebook_login\n\nnotebook_login()'

In [52]:
'''from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-amazon-en-es",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)'''

'from transformers import Seq2SeqTrainingArguments\n\nbatch_size = 8\nnum_train_epochs = 8\n# Show the training loss with every epoch\nlogging_steps = len(tokenized_datasets["train"]) // batch_size\nmodel_name = model_checkpoint.split("/")[-1]\n\nargs = Seq2SeqTrainingArguments(\n    output_dir=f"{model_name}-finetuned-amazon-en-es",\n    evaluation_strategy="epoch",\n    learning_rate=5.6e-5,\n    per_device_train_batch_size=batch_size,\n    per_device_eval_batch_size=batch_size,\n    weight_decay=0.01,\n    save_total_limit=3,\n    num_train_epochs=num_train_epochs,\n    predict_with_generate=True,\n    logging_steps=logging_steps,\n    push_to_hub=True,\n)'

In [53]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-amazon-en-es",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
)

In [54]:
'''import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}'''

'import numpy as np\n\n\ndef compute_metrics(eval_pred):\n    predictions, labels = eval_pred\n    # Decode generated summaries into text\n    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)\n    # Replace -100 in the labels as we can\'t decode them\n    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)\n    # Decode reference summaries into text\n    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)\n    # ROUGE expects a newline after each sentence\n    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]\n    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]\n    # Compute ROUGE scores\n    result = rouge_score.compute(\n        predictions=decoded_preds, references=decoded_labels, use_stemmer=True\n    )\n    # Extract the median scores\n    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}\n    return {k: round(v, 4) for k

In [55]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [56]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [57]:
tokenized_datasets = tokenized_datasets.remove_columns(
    language_dataset["train"].column_names
)

In [58]:
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[   486,   2821,   9469,  10832,    304,    287,  19521,    264,  47842,
            297,      1,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0],
        [   259, 185343,    307, 130457,    345, 108496,  36264,    281,   4054,
            356,  27402,    259,   9877,    287,    814,   7225,  15344,    259,
          87470,    267,  19466,    304,    935,    264,  85692,   3042,   3362,
           4751,   1521,   2161,   4911,    274,  82273,    271,  15924,   1300,
            305,  17205,  39687,    259,  96653,      1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [59]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [60]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


TrainOutput(global_step=4864, training_loss=2.077551581406672, metrics={'train_runtime': 1434.848, 'train_samples_per_second': 27.091, 'train_steps_per_second': 3.39, 'total_flos': 3368127269191680.0, 'train_loss': 2.077551581406672, 'epoch': 8.0})

In [61]:
trainer.evaluate()

{'eval_loss': 0.8496327996253967,
 'eval_rouge1': 73.0353,
 'eval_rouge2': 38.212,
 'eval_rougeL': 73.0714,
 'eval_rougeLsum': 72.9993,
 'eval_runtime': 24.5053,
 'eval_samples_per_second': 56.6,
 'eval_steps_per_second': 7.101,
 'epoch': 8.0}

In [62]:
tokenized_datasets.set_format("torch")

In [63]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [65]:
from torch.utils.data import DataLoader

batch_size = 8
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], collate_fn=data_collator, batch_size=batch_size
)

In [66]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [67]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [68]:
from transformers import get_scheduler

num_train_epochs = 10
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [69]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # ROUGE expects a newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [70]:
'''from huggingface_hub import get_full_repo_name

model_name = "test-bert-finetuned-squad-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name'''

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 4>:4                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:118 in _inner_fn    │
│                                                                                                  │
│   115 │   │   if check_use_auth_token:                                                           │
│   116 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   117 │   │                                                                                      │
│ ❱ 118 │   │   return fn(*args, **kwargs)                                                         │
│   119 │                                                                                          │
│   120 │   return _inner_fn  # type: ignore                                                       │
│   121                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/hf_api.py:3933 in get_full_repo_name     │
│                                                                                                  │
│   3930 │   │   │   if "/" in model_id:                                                           │
│   3931 │   │   │   │   username = model_id.split("/")[0]                                         │
│   3932 │   │   │   else:                                                                         │
│ ❱ 3933 │   │   │   │   username = self.whoami(token=token)["name"]  # type: ignore               │
│   3934 │   │   │   return f"{username}/{model_id}"                                               │
│   3935 │   │   else:                                                                             │
│   3936 │   │   │   return f"{organization}/{model_id}"                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:118 in _inner_fn    │
│                                                                                                  │
│   115 │   │   if check_use_auth_token:                                                           │
│   116 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   117 │   │                                                                                      │
│ ❱ 118 │   │   return fn(*args, **kwargs)                                                         │
│   119 │                                                                                          │
│   120 │   return _inner_fn  # type: ignore                                                       │
│   121                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/hf_api.py:925 in whoami                  │
│                                                                                                  │
│    922 │   │   """                                                                               │
│    923 │   │   r = get_session().get(                                                            │
│    924 │   │   │   f"{self.endpoint}/api/whoami-v2",                                             │
│ ❱  925 │   │   │   headers=self._build_hf_headers(                                               │
│    926 │   │   │   │   # If `token` is provided and not `No

In [71]:
'''from huggingface_hub import Repository

output_dir = "results-mt5-finetuned-squad-accelerate"
repo = Repository(output_dir, clone_from=repo_name)'''

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 4>:4                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'repo_name' is not defined

In [74]:
from tqdm.auto import tqdm
import torch
import numpy as np

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch["labels"]

            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(
                batch["labels"], dim=1, pad_index=tokenizer.pad_token_id
            )

            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()

            # Replace -100 in the labels as we can't decode them
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            decoded_preds = tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True
            )
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            decoded_preds, decoded_labels = postprocess_text(
                decoded_preds, decoded_labels
            )

            rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)

    # Compute metrics
    result = rouge_score.compute()
    # Extract the median ROUGE scores
    result = {key: value * 100 for key, value in result.items()}
    result = {k: round(v, 4) for k, v in result.items()}
    print(f"Epoch {epoch}:", result)
'''
    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )'''

  0%|          | 0/6080 [00:00<?, ?it/s]

Epoch 0: {'rouge1': 2.8367, 'rouge2': 0.6468, 'rougeL': 2.8597, 'rougeLsum': 2.7988}
Epoch 1: {'rouge1': 2.8679, 'rouge2': 0.2259, 'rougeL': 2.8809, 'rougeLsum': 2.8552}
Epoch 2: {'rouge1': 26.2796, 'rouge2': 25.2902, 'rougeL': 26.2549, 'rougeLsum': 26.192}
Epoch 3: {'rouge1': 30.6448, 'rouge2': 30.2091, 'rougeL': 30.6643, 'rougeLsum': 30.6324}
Epoch 4: {'rouge1': 30.5479, 'rouge2': 30.2091, 'rougeL': 30.5407, 'rougeLsum': 30.5479}
Epoch 5: {'rouge1': 30.5696, 'rouge2': 30.2091, 'rougeL': 30.5696, 'rougeLsum': 30.584}
Epoch 6: {'rouge1': 30.6417, 'rouge2': 30.2091, 'rougeL': 30.6561, 'rougeLsum': 30.6128}
Epoch 7: {'rouge1': 30.6056, 'rouge2': 30.2091, 'rougeL': 30.6128, 'rougeLsum': 30.6056}
Epoch 8: {'rouge1': 30.6056, 'rouge2': 30.2091, 'rougeL': 30.6128, 'rougeLsum': 30.6056}
Epoch 9: {'rouge1': 30.6056, 'rouge2': 30.2091, 'rougeL': 30.6128, 'rougeLsum': 30.6056}


'\n    # Save and upload\n    accelerator.wait_for_everyone()\n    unwrapped_model = accelerator.unwrap_model(model)\n    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)\n    if accelerator.is_main_process:\n        tokenizer.save_pretrained(output_dir)\n        repo.push_to_hub(\n            commit_message=f"Training in progress epoch {epoch}", blocking=False\n        )'

In [75]:
from transformers import pipeline

hub_model_id = "huggingface-course/mt5-small-finetuned-amazon-en-es"
summarizer = pipeline("summarization", model=hub_model_id)

In [1]:
def print_summary(idx):
    sentence = language_dataset["test"][idx]["sentence"]
    data = language_dataset["test"][idx]["data"]
    summary = summarizer(language_dataset["test"][idx]["sentence"])[0]["summary_text"]
    print(f"'>>> Sentence: {sentence}'")
    print(f"\n'>>> Title: {data}'")
    print(f"\n'>>> Summary: {summary}'")

In [77]:
print_summary(100)

Your max_length is set to 20, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


'>>> Review: Pneumonía COVID-19: hallazgos en serie de TC'

'>>> Title: COVID-19'

'>>> Summary: Libro de noticias'


In [2]:
print_summary(0)

NameError: ignored